In [21]:
# load credentials from environment variables
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [25]:

for band in ['B02']:
    !rm -r *.zarr
    %time !xcube sh gen S2L2A -o ooe_{band}.zarr -b {band} -g 14.318395625014208,48.359153946428485,14.506361110382839,48.543395524629055 -r 0.000134 -t 2018-01-01,2018-01-31 -p 2W 2>/dev/null

Writing cube to ooe_B02.zarr...
Cube written to ooe_B02.zarr, took 6.20 seconds.
CPU times: user 132 ms, sys: 21.4 ms, total: 154 ms
Wall time: 8.03 s
